In [1]:
import pandas as pd

In [2]:
annot_tweets = pd.read_parquet('annotated_tweets.parquet')
annot_tweets.shape

(600, 24)

In [3]:
annot_tweets.head(10)

,id,tweet,highlight,not_good,polarized,tweet_bias,toxic_language,toxic_language_txt,hate_speech,hate_speech_txt,...,dehumanization,dehumanization_txt,crime_imputation,crime_imputation_txt,divisive_language,divisive_language_txt,strong_bias,strong_bias_txt,other,other_txt
0,1,"“PERDEU, MANÉ”! Foi isso mesmo que o ministro ...",1,0,1,1,0,,0,,...,0,,0,,0,,0,,0,
1,2,É fundamental q toda pessoa q está sendo imped...,1,0,1,-1,0,,0,,...,0,,0,,1,diferente do lado de lá,0,,0,
2,3,🚨 URGENTE! O diretor bolsonarista da Polícia R...,1,0,1,-1,0,,0,,...,0,,1,ISSO É UM GOLPE DE ESTADO! #Eleicoes2022,1,O diretor bolsonarista,0,,0,
3,4,"🚨URGENTE: Quer dizer que os ""patriotas"" iriam ...",1,0,1,1,0,,0,,...,1,infiltrados; terroristas,1,"infiltrados, terroristas, antifas, black block...",0,,0,,0,
4,5,Se você roubasse uma eleição em favor de um cr...,1,0,1,1,0,,0,,...,0,,1,Se você roubasse uma eleição em favor de um cr...,0,,0,,0,
5,6,"Renata Vasconcellos: ""Em época de pandemia, o ...",1,0,0,0,0,,0,,...,0,,0,,0,,0,,0,
6,7,Vamos simplificar... Poste aqui nesse tweet TO...,1,0,1,-1,0,,0,,...,0,,0,,0,,1,Bora criar uma listinha pra facilitar os traba...,0,
7,8,O mínimo que deve acontecer depois do golpismo...,1,0,1,-1,0,,0,,...,0,,1,Responsabilização da Jovem Pan,0,,0,,0,
8,9,🚨URGENTE!🚨Cacique Xavante Tserere ACABA DE SER...,0,0,1,1,0,,0,,...,0,,0,,1,Uma afronta direta ao presidente e uma escalad...,1,Uma afronta direta ao presidente e uma escalad...,0,
9,10,"No Nordeste, a Polícia Rodoviária Federal desc...",1,0,1,-1,0,,0,,...,0,,1,Polícia Rodoviária Federal descumpre a ordem j...,0,,0,,0,


In [4]:
annot_tweets.shape

(600, 24)

In [5]:
annot_tweets.drop_duplicates(subset='tweet', ignore_index=True).shape

(600, 24)

In [21]:
# checking for updates

template = (
    '{} -> {}\n'
    '{}: {}\n'
    '{}: {}\n\n'
)

for idx, row in annot_tweets.iterrows():
    for idx_2, row_2, in annot_tweets.iterrows():
        if idx_2 > idx:
            if row['tweet'][0:40] == row_2['tweet'][0:40]:
                print(template.format(idx, idx_2, idx, row['tweet'], idx_2, row_2['tweet']))

In [ ]:
left_tweets = annot_tweets[annot_tweets['tweet_bias']==-1].sample(45)
left_tweets.shape

In [ ]:
right_tweets = annot_tweets[annot_tweets['tweet_bias']==1].sample(45)
right_tweets.shape

In [ ]:
random_sample = pd.concat([left_tweets, right_tweets]).sample(frac=1)
random_sample.reset_index(inplace=True, drop=True)
random_sample.shape

In [ ]:
random_sample.head()

In [ ]:
def define_bias(bias_val):
    if bias_val == -1:
        return 'L'
    elif bias_val == 0:
        return 'C'
    elif bias_val == 1:
        return 'R'
    else:
        return 'U'

In [ ]:
tweets = []
for idx, row in random_sample.iterrows():
    id = define_bias(row['tweet_bias'])+str(idx).zfill(3)
    tweets.append({
        'tweet_id': id,
        'tweet_text': row['tweet'],
        'bias': define_bias(row['tweet_bias']),
        'tweet_bias': row['tweet_bias']
    })

In [ ]:
indexed_tweets = pd.DataFrame.from_records(tweets)

In [ ]:
indexed_tweets.head()

In [ ]:
human = [
    indexed_tweets[indexed_tweets['bias']=='L'][0:15],
    indexed_tweets[indexed_tweets['bias']=='R'][0:15]
]
machine = [
    indexed_tweets[indexed_tweets['bias']=='L'][15:30],
    indexed_tweets[indexed_tweets['bias']=='R'][15:30]
]
placebo = [
    indexed_tweets[indexed_tweets['bias']=='L'][30:45],
    indexed_tweets[indexed_tweets['bias']=='R'][30:45]
]

human_tts = pd.concat(human)
machine_tts = pd.concat(machine)
placebo_tts = pd.concat(placebo)

In [ ]:
human_tts['rephrased_id'] = human_tts['tweet_id'].apply(lambda x: 'H'+x)
machine_tts['rephrased_id'] = machine_tts['tweet_id'].apply(lambda x: 'M'+x)
placebo_tts['rephrased_id'] = placebo_tts['tweet_id'].apply(lambda x: 'P'+x)

human_tts['rephrased_text'] = human_tts['tweet_text'].apply(lambda x: 'Rep - '+x)
machine_tts['rephrased_text'] = machine_tts['tweet_text'].apply(lambda x: 'Rep - '+ x)
placebo_tts['rephrased_text'] = placebo_tts['tweet_text'].apply(lambda x: 'Rep - '+ x)

human_tts['group'] = 'human'
machine_tts['group'] = 'machine'
placebo_tts['group'] = 'placebo'

tweets_df = pd.concat([human_tts, machine_tts, placebo_tts]).sample(frac=1).reset_index(drop=True)
tweets_df.shape

In [ ]:
tweets_df.sample(10)

In [ ]:
tweets_df.to_parquet('tweets.parquet', index=False)